In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import numpy as np
import datetime
import time
import sys
import os
import psycopg2
from psycopg2.extras import RealDictCursor

def get_data(src_str,basedate,output_file):
    startDate = basedate[:6]
    endDate = datetime.date.today().strftime('%Y%m')
    
    headers = {'Referer': 'https://datalab.naver.com/'
              ,'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0'}
    url = 'http://datalab.naver.com/qcHash.naver'
    formdata={'queryGroups': src_str
              , 'startDate': startDate, 'endDate': endDate, 'timeUnit': 'week'
              , 'gender': '', 'age': '','device': ''}
    
    r=requests.post(url,headers=headers,data=formdata)
    hash_data = json.loads(r.text)
    hashKey = hash_data['hashKey']
    #print(hash_data)
    
    trend_url = f'http://datalab.naver.com/keyword/trendResult.naver?hashKey={hashKey}'
    #print(trend_url)
    r=requests.get(trend_url,headers=headers)
    soup = bs(r.text,'lxml')
    graph_raw = soup.find('div',id = 'graph_data').get_text()
    #graph = json.loads(soup.find('div',id = 'graph_data').get_text())
    with open(f'{grp1_path}/{output_file}','a',encoding='utf-8') as f:
        f.write(graph_raw+'\n')
    time.sleep(0.5)


In [2]:
df= pd.read_csv('new_keyword_20210112_backdaeun.txt', header=None)
rl=df[0].to_list()
grp1_path = './result_data/grp_new'
total_key_cnt = len(rl)
#total_key_cnt=30
work_key_cnt = 0

cnt=0
req_cnt=0
basedate='20170102'
today=datetime.datetime.now().strftime('%Y%m%d')
print(today)
output_file=f'keyword_crawling_{today}.txt'
src_str='아파트__SZLIG__아파트__OUML__'

for i in range(work_key_cnt,total_key_cnt):
    #src_str='이월드__SZLIG__이월드__OUML__'
    #key_grp=rl[i].decode("utf-8").strip().split(',')
    #grp=key_grp[0]
    #key=str.lower(key_grp[1])
    key=rl[i]
    src_str+=key+'__SZLIG__'+key+'__OUML__'
    # 키워드 리스트의 마지막 항목 이거나 그룹핑 최대 횟수에 도달할경우 request처리
    if i==len(rl)-1 or cnt==3:
        #print(src_str)
        req_cnt += 1
        cnt=0
        src_str = src_str[:-8]
        #print(work_key_cnt,total_key_cnt,req_cnt,src_str)
        get_data(src_str,basedate,output_file)
        src_str='아파트__SZLIG__아파트__OUML__'
        
        sys.stdout.write(f'\r | {work_key_cnt} / {total_key_cnt} |  /  | {req_cnt}    |{src_str} ')
        
        sys.stdout.flush()
    work_key_cnt +=1
    cnt+=1


20210113
 | 3225 / 3226 |  /  | 1075    |아파트__SZLIG__아파트__OUML__ 

In [ ]:
df[0].to_list

In [58]:
try:
    conn = psycopg2.connect(host='10.123.184.193', dbname='edw', user='sys', password='eos40d')
    cur = conn.cursor(cursor_factory=RealDictCursor)
except Exception:
    print('Fail to connect DB')
    
###1st 계산을 위한 존데이터 기준날짜 20191104의 scale조회하기 df2

sql = "select * from sbd.sbds_search_cnt_week where start_date = '20191104'"
cur.execute(sql)
result = cur.fetchall()
df2 = pd.DataFrame(result)
df2=df2.rename(columns={'start_date':'stand_start_date','scale':'stand_scale' })

In [61]:
df3=df2[df2['stand_scale']>0]

In [68]:

a=df3['stand_scale'].min()
b=df3['stand_scale'].max()
c=df3['stand_scale'].median()
d=df3['stand_scale'].mean()

In [69]:
a,b,c,d

(1, 5714805, 178.0, 1164.476865526012)

In [79]:
df2[df2['keyword']=='소']

,keyword,stand_start_date,end_date,year_month,stand_scale,indate
230915,소,2019-11-04,2019-11-10,201911,5574,2019-11-11


In [87]:
df2[df2[(['stand_scale']>=10000) and (df2['stand_scale']<=11000)]

SyntaxError: unexpected EOF while parsing (<ipython-input-87-b3974baea854>, line 1)

In [92]:
(df2['stand_scale']>10000) &(df2['stand_scale']<=11000)

0         False
1         False
2         False
3         False
4         False
          ...  
453412    False
453413    False
453414    False
453415    False
453416    False
Name: stand_scale, Length: 453417, dtype: bool

In [94]:
df4=df2[(df2['stand_scale']>10000) &(df2['stand_scale']<=11000)]

In [95]:
df4['keyword']

49              이지아
976        영어공부혼자하기
2239          남자목폴라
2268            알파카
2574           한국지도
            ...    
433431          아파트
433668       빈센트반고흐
437107           링티
445383       표고버섯요리
446815    시크릿쥬쥬별의여신
Name: keyword, Length: 451, dtype: object